Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import os
import numpy as np
import random
import string
import tensorflow as tf
import urllib
import zipfile
import time
import re
%matplotlib inline

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print 'Found and verified', filename
  else:
    print statinfo.st_size
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  print f.namelist()
  for name in f.namelist():
    return f.read(name)
  f.close()
  
text = read_data(filename)
print "Data size", len(text)

['text8']
Data size 100000000


In [4]:
# look at some text
print text[:100]
print text[-100:]

 anarchism originated as a term of abuse first used against early working class radicals including t
identified in one eight four two and extensively excavated in one nine six three one nine six five b


In [4]:
# separate text into words
words = text.split()
print 'Data size', len(words)

Data size 17005207


### Create a small validation set.

In [15]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print train_size, train_text[:64]
print valid_size, valid_text[:64]

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


### Utility functions to map characters to vocabulary IDs and back.

In [668]:
' ' + string.ascii_lowercase

' abcdefghijklmnopqrstuvwxyz'

In [16]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])  # ordinal of letter 'a'

def char2id(char):
  if char in string.ascii_lowercase:  # if detect a lowercase letter
    return ord(char) - first_letter + 1  # return value 1-26
  elif char == ' ':
    return 0
  else:
    print 'Unexpected character:', char
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)  # return [a-z]
  else:
    return ' '

print char2id('a'), char2id('z'), char2id(' '), char2id('ï')
print('-'*16)
print id2char(1), id2char(26), id2char(0)

1 26 0 Unexpected character: ï
0
----------------
a z  


### Function to generate a training batch for the LSTM model.

In [24]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size / batch_size
    # list of offsets within batch
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in xrange(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0  # get id of a char
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size  # move cursor
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())  # add id of char for 1 to num_unrollings
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation.
  """
  return [id2char(c) for c in np.argmax(probabilities, 1)]  # get char of an id

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation.
  """
  #print 'Batch shape: {}'.format(batches[0].shape)
  #print 'First batch, First char: {}'.format(batches[0][0])
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

### Generate batches

In [7]:
# training and validation batches
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1) # returns batch size 1, +1 unrolling 

# look at the text from various segments
segment_look = 0
num_char = 64
show = segment_look * len(train_text)/batch_size
print "index {} to {}:\n{}".format(show, show+num_char, train_text[show:show+num_char])
print('-'*16)

print batches2string(train_batches.next())
print batches2string(train_batches.next())
print('-'*16)
print batches2string(valid_batches.next())
print batches2string(valid_batches.next())

index 0 to 64:
ons anarchists advocate social relations based upon voluntary as
----------------
['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 't

### Functions for predictions

In [18]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in xrange(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, axis=1)[:,None]

### Simple LSTM Model
http://colah.github.io/posts/2015-08-Understanding-LSTMs/  

<img width="60%" src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png">

In [173]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  ## Parameters:
  # Input (Write) gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Forget (Keep) gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Memory cell: input, state and bias.        
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Output (Read) gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """
    Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates.
    """
    # what to keep (1) or forget (0) from cell state
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    
    # new info to store in cell state
    # values to update
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    # new candidate values to add to state
    update = tf.tanh(tf.matmul(i, cx) + tf.matmul(o, cm) + cb)
    
    # update old cell state C[t-1] into new cell state C[t]
    state = (forget_gate * state) + (input_gate * update)
    
    # decide the output
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    h = output_gate * tf.tanh(state)
    return h, state

  # Input data.
  train_data = list()
  for _ in xrange(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=False)  ## orig 10.0, 5000, 0.1, True
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

### Run the LSTM

In [174]:
num_steps = 7001  ## orig 7001
summary_frequency = 100

t0 = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized\n=========='
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % (5.*summary_frequency) == 0:  ## orig 2.5*summary_frequency
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step ', step, '=', mean_loss, '\nlearning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))
      print '-' * 30
# show how much time elapsed
print (time.time()-t0)/60., 'minutes elapsed'

Initialized
Average loss at step  0 = 3.29952096939 
learning rate: 10.0
Minibatch perplexity: 27.10
cmmwuj  owrqyf oe a vb ayeotjtp jsagtecex w ep gapegmss doi nspzod pxy    lte fh
bxxzosqroolkir nzk dbuoo j bdl  uik yio anaqi  gp kfcjq s csrq zwygdnrgk uehp yi
ui d dt niegm nlifhtgirrqr n ophpgeei iapfhndstsfpvqemtasstzd mguwbod mm gewwt v
neookwmvyl tcbmz d fpyofbndlkxywzjki  wfjnpzwcla axdqlkta nn e ioutzala vuwenekh
fjizkt xy  o bubsnvvl it r uvsdnfss eki eulhwi emosavt epjo hmn  rmsnchuuchkmlqh
Validation set perplexity: 19.94
------------------------------
Average loss at step  500 = 10.8983187985 
learning rate: 7.94328
Minibatch perplexity: 6.51
Validation set perplexity: 7.28
------------------------------
Average loss at step  1000 = 9.28752010226 
learning rate: 6.30957
Minibatch perplexity: 7.06
dites of the mous hurhseed events fiel witete that rebat wore sueflo ont s to bi
y of tradcons thit the arebuti doobas sypcess the iting be vorte world and hairs
chipit sintion wir

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.


Cell state (akin to a conveyor belt):  
http://colah.github.io/posts/2015-08-Understanding-LSTMs/  
<img width="70%" src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-C-line.png"></a>


1. Forget gate:  
<img width="70%"src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png'>

2. New info to store in cell state (Input gate * New candidate values:    
<img width="70%"src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png'>

3. Update the old cell state, C[t−1], into the new cell state C[t]:  
<img width="70%"src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png'>

4. Decide output (Output gate * cell state C[t]):  
<img width="70%"src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png'>

---

### Use single matrix multiplication for LSTM cell

In [203]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  ## Parameters:
  # combined f,i,c,o
  ifco_x = tf.Variable(tf.truncated_normal([4, vocabulary_size, num_nodes], -0.1, 0.1))
  print ifco_x.get_shape().as_list()
  ifco_m = tf.Variable(tf.truncated_normal([4, num_nodes, num_nodes], -0.1, 0.1))
  ifco_b = tf.Variable(tf.zeros([4, 1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """
    Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates.
    """                   
    i_list = tf.pack([i, i, i, i])
    #print i_list.get_shape().as_list()
    o_list = tf.pack([o, o, o, o])
                          
    ins = tf.batch_matmul(i_list, ico_x)
    outs = tf.batch_matmul(o_list, ifco_m)
    
    h_x = ins + outs + ifco_b
    #print h_x.get_shape().as_list()
    
    #forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    forget_gate = tf.sigmoid(h_x[1,:,:])
    
    #input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    input_gate = tf.sigmoid(h_x[0,:,:])
    
    #update = tf.tanh(tf.matmul(i, cx) + tf.matmul(o, cm) + cb)
    update = tf.tanh(h_x[2,:,:])
    
    state = forget_gate*state + input_gate*update
    
    #output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    output_gate = tf.sigmoid(h_x[3,:,:])
    
    h = output_gate * tf.tanh(state)
    #print 'h', h.get_shape().as_list()
    return h, state

  # Input data.
  train_data = list()
  for _ in xrange(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=False)  ## orig 10.0, 5000, 0.1, True
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

[4, 27, 64]


### Run it with single matrix

In [172]:
num_steps = 7001  ## orig 7001
summary_frequency = 100

t0 = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized\n=========='
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % (5.*summary_frequency) == 0:  ## orig 2.5*summary_frequency
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step ', step, '=', mean_loss, '\nlearning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))
      print '-' * 30
# show how much time elapsed
print (time.time()-t0)/60., 'minutes elapsed'

Initialized
Average loss at step  0 = 3.2942814827 
learning rate: 10.0
Minibatch perplexity: 26.96
ylnonkxvt nkueilucjf w h yuamjzsldesyid  lhumkeyfhdiafedazndceszgxxpjhk enfjbmii
g dtnffulibkn tqivpe poenheqabvnjz rux yiegucqylhfhxdaenevtatngejghspf nnfr  e e
hx ntkifs  nauohgqebejeo lueyiluyutemqe  y ceefkrvjopd se u p aeou  ehn ghl  imn
r seowdsgbnwtfveafyoenveejetrltlvg  kcdtfurnh n a q  cfn thjt yuudx oxkos rchshr
f  ymmuhwczueu  hjjpkfpd ndn inenjspwh kvdlkftsephmkijcmsne sa cwezuiystwepo w  
Validation set perplexity: 20.31
------------------------------
Average loss at step  500 = 10.8926817107 
learning rate: 7.94328
Minibatch perplexity: 7.35
Validation set perplexity: 7.33
------------------------------
Average loss at step  1000 = 9.33507280588 
learning rate: 6.30957
Minibatch perplexity: 6.45
ing he bust canss is mad polmums for argerarion westrada for ong nytuca reigatio
ap waillaka humbast play of sua a andaning bearwied motorners stologed coveline 
 of nutioning sargi

## Variants on LSTM 

### Peephole connections:  
ftp://ftp.idsia.ch/pub/juergen/TimeCount-IJCNN2000.pdf  
<img width='80%' src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-peepholes.png'>

### Gated Recurrent Unit (GRU):  
http://arxiv.org/pdf/1406.1078v3.pdf  
- combine forget and input gates
- merge cell and hidden states  
<img width='80%' src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png'>


### Attention
http://arxiv.org/pdf/1502.03044v2.pdf
>Let every step of an RNN pick information to look at from some larger collection of information. For example, if you are using an RNN to create a caption describing an image, it might pick a part of the image to look at for every word it outputs



---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

### Dictionary of bigrams

In [9]:
# build dictionary of bigrams
dictionary = dict()
count = 0
for i in ' ' + string.ascii_lowercase:
    for j in ' ' + string.ascii_lowercase:
        dictionary[i+j] = count
        count += 1
print len(dictionary)

# build reverse dictionary of bigrams
reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
print len(reverse_dictionary)

729
729


### Function to generate a training batch for embedded bigrams

In [10]:
class BatchGeneratorBigram(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size / batch_size
    # list of offsets within batch
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size), dtype=np.int)  # id of char to be embedded
    for b in xrange(self._batch_size):
      #batch[b] = char2id(self._text[self._cursor[b]])  # use w/ character model
      c1 = self._text[self._cursor[b]] # 1st char of bigram
      c2 = self._text[(self._cursor[b] + 1) % self._text_size] # 2nd char of bigram
      batch[b] = dictionary[c1+c2]
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size  # move cursor
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())  # add id of char for 1 to num_unrollings
    self._last_batch = batches[-1]
    return batches

def bigrambatches2string(batches):
  """Convert a sequence of batches back into string
  representation.
  """
  s = [''] * batches[0].shape[0]
  for b in batches:
    #s = [''.join(x) for x in zip(s, [id2char(c) for c in b])]  # use w/ character model
    #b = [re.findall(r'(?<=99)[12]?[0-9](?=99)', str(a)) for a in b]
    #s = [''.join(x) for x in zip(s, [id2char(int(c[0])) for c in b])]  # use w/ bigram model
    s = [''.join(x) for x in zip(s, [reverse_dictionary[c][0] for c in b])]  # use w/ character model
  return s

### Generate training, validation batches for embedded bigrams

In [13]:
# training and validation batches
batch_size = 64
num_unrollings = 11
train_batches = BatchGeneratorBigram(train_text, batch_size, num_unrollings)
valid_batches = BatchGeneratorBigram(valid_text, 1, 2) # returns batch size 1, +2 unrolling
train_batches_y = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches_y = BatchGenerator(valid_text, 1, 2) # returns batch size 1, +2 unrolling 

# look at the text from various segments
segment_look = 0
show = segment_look * len(train_text)/batch_size
print "index {} to {}:\n{}".format(show, show+80, train_text[show:show+64])
print('-'*16)

print train_batches.next()[0].shape
print bigrambatches2string(train_batches.next())
print('-'*16)
print valid_batches.next()
print valid_batches_y.next()
print bigrambatches2string(valid_batches.next())

index 0 to 80:
ons anarchists advocate social relations based upon voluntary as
----------------
(64,)
['sts advocate', 'ry governmen', 'es national ', ' monasteries', 'aca princess', 'hard baer h ', 'gical langua', 'or passenger', 'he national ', 'ook place du', 'her well kno', 'even six sev', 'th a gloss c', 'obably been ', 'o recognize ', 'eived the fi', 'cant than in', 'itic of the ', 'ght in signs', ' uncaused ca', 'lost as in d', 'ellular ice ', ' size of the', 'him a stick ', 'rugs confusi', 'take to comp', 'the priest o', 'm to name it', ' barred atte', 'tandard form', 'such as esot', 'e on the gro', ' of the orig', ' hiver one n', ' eight march', 'he lead char', 's classical ', 'e the non gm', 'l analysis f', 'ormons belie', ' or at least', 'disagreed up', 'ng system ex', 'types based ', 'nguages the ', ' commission ', 'ss one nine ', 'ux suse linu', 'the first da', 'i concentrat', 'society nehr', 'latively sti', 'tworks sharm', 'r hirohito t', 'itical initi', ' most of the', 

### Build the bigram graph with embeddings

In [863]:
num_nodes_1 = 128
num_nodes = 32
embedding_size = 128 # Dimension of the embedding vector.

graph = tf.Graph()
with graph.as_default():
  
  ## Parameters: (same as i,f,g,o)
  ifco_x = tf.Variable(tf.truncated_normal([4, embedding_size, num_nodes_1], -0.1, 0.1))
  print ifco_x.get_shape().as_list()
  ifco_m = tf.Variable(tf.truncated_normal([4, num_nodes_1, num_nodes_1], -0.1, 0.1))
  ifco_b = tf.Variable(tf.zeros([4, 1, num_nodes_1]))
    
  ifco_x2 = tf.Variable(tf.truncated_normal([4, num_nodes_1, num_nodes], -0.1, 0.1))
  ifco_m2 = tf.Variable(tf.truncated_normal([4, num_nodes, num_nodes], -0.1, 0.1))
  ifco_b2 = tf.Variable(tf.zeros([4, 1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes_1]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes_1]), trainable=False)
  saved_output2 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state2 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
    
  # Embedding Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size**2, embedding_size], -1.0, 1.0), trainable=False)
  
  # Dropout
  #keep_prob = tf.constant(0.7)
  keep_prob = tf.placeholder(tf.float32)

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """
    Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates.
    """                   
    i_list = tf.pack([i, i, i, i])
    o_list = tf.pack([o, o, o, o])
                          
    ins = tf.batch_matmul(i_list, ifco_x)
    outs = tf.batch_matmul(o_list, ifco_m)
    
    h_x = ins + outs + ifco_b

    forget_gate = tf.sigmoid(h_x[1,:,:])

    input_gate = tf.sigmoid(h_x[0,:,:])
    input_gate_d = tf.nn.dropout(input_gate, keep_prob)  # dropout input
    update = tf.tanh(h_x[2,:,:])
    state = forget_gate*state + input_gate_d*update
    #state_d = forget_gate*state + input_gate_d*update
    
    output_gate = tf.sigmoid(h_x[3,:,:])
    output_gate_d = tf.nn.dropout(output_gate, keep_prob)  # dropout input
    
    h = output_gate_d * tf.tanh(state)
    #h_pred = output_gate * tf.tanh(state)
    return h, state#, h_pred  # dont use dropout for predictions

  def lstm_cell_2(i, o, state):         
    i_list = tf.pack([i, i, i, i])
    o_list = tf.pack([o, o, o, o])                      
    ins = tf.batch_matmul(i_list, ifco_x2)
    outs = tf.batch_matmul(o_list, ifco_m2)
    h_x = ins + outs + ifco_b2
    
    forget_gate = tf.sigmoid(h_x[1,:,:])
    input_gate = tf.sigmoid(h_x[0,:,:])
    #input_gate_d = tf.nn.dropout(input_gate, keep_prob)  # dropout input
    update = tf.tanh(h_x[2,:,:])
    state = forget_gate*state + input_gate*update
    #state_d = forget_gate*state + input_gate_d*update
    
    output_gate = tf.sigmoid(h_x[3,:,:])
    #output_gate_d = tf.nn.dropout(output_gate, keep_prob)  # dropout input
    
    h = output_gate * tf.tanh(state)
    #h_pred = output_gate * tf.tanh(state)
    return h, state#, h_pred  # dont use dropout for predictions

  # Input data.
  train_data = list()
  train_data_y = list()
  for _ in xrange(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))  # removed ohe of char
    train_data_y.append(
      tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))  # uses ohe of char
  train_labels = train_data_y[2:]  # offset labels for bigram input
  
  # Embedded input data
  encoded_inputs = list()
  for bigram in train_data:
    embed = tf.nn.embedding_lookup(embeddings, bigram)
    encoded_inputs.append(embed)
  train_inputs = encoded_inputs[:num_unrollings-1]

  # Unrolled LSTM loop.
  outputs = list()
  #out_preds = list()
  output = saved_output
  output2 = saved_output2
  state = saved_state
  state2 = saved_state2
  for i in train_inputs:
    #output, state, out_pred = lstm_cell(i, output, state)
    output, state = lstm_cell(i, output, state)
    #output2, state2, _ = lstm_cell_2(output, output2, state2)
    #_, _, out_pred2 = lstm_cell_2(out_pred, output2, state2)
    output2, state2, = lstm_cell_2(output, output2, state2)
    outputs.append(output2)
    #out_preds.append(out_pred2)
  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state),
                                saved_output2.assign(output2),
                                saved_state2.assign(state2)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    #logits_pred = tf.nn.xw_plus_b(tf.concat(0, out_preds), w, b)
    print 'logits', logits.get_shape().as_list()
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))
    print 'labels', tf.concat(0, train_labels).get_shape().as_list()

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 4000, 0.1, staircase=False)  ## orig 10.0, 5000, 0.1, True
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  #train_prediction = tf.nn.softmax(logits_pred)
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1]) # removed ohe of char
  sample_input_emb = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes_1]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes_1]))
  saved_sample_output2 = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state2 = tf.Variable(tf.zeros([1, num_nodes]))

  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes_1])),
    saved_sample_state.assign(tf.zeros([1, num_nodes_1])),
    saved_sample_output2.assign(tf.zeros([1, num_nodes])),
    saved_sample_state2.assign(tf.zeros([1, num_nodes]))
    )

  sample_output, sample_state = lstm_cell(
      sample_input_emb, saved_sample_output, saved_sample_state)
  sample_output2, sample_state2 = lstm_cell_2(
      sample_output, saved_sample_output2, saved_sample_state2)

  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state),
                                saved_sample_output2.assign(sample_output2),
                                saved_sample_state2.assign(sample_state2)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output2, w, b))

[4, 128, 128]
logits [640, 27]
labels [640, 27]


### Run it with bigrams

In [864]:
# training and validation batches
train_batches = BatchGeneratorBigram(train_text, batch_size, num_unrollings)
valid_batches = BatchGeneratorBigram(valid_text, 1, 2) # returns batch size 1, +2 unrolling
train_batches_y = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches_y = BatchGenerator(valid_text, 1, 2) # returns batch size 1, +2 unrolling 

num_steps = 7001  ## orig 7001
summary_frequency = 100

t0 = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized\n=========='
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    batches_y = train_batches_y.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]  # feed input data
      feed_dict[train_data_y[i]] = batches_y[i]  # feed vectorized label data
    feed_dict[keep_prob] = 0.5  # dropout during training
    
    # evaluate graph
    #_, l, predictions, lr = session.run(
    #  [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    _, l, lr = session.run(
      [optimizer, loss, learning_rate], feed_dict=feed_dict)
    
    mean_loss += l
    if step % (5.*summary_frequency) == 0:  ## orig 2.5*summary_frequency
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step', step, '=', mean_loss, '\nlearning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches_y)[2:])  # offset labels for bigram
      feed_dict[keep_prob] = 1.
      predictions = train_prediction.eval(feed_dict=feed_dict)  # predict w/out dropout
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          #feed = np.random.randint(27, size=[1])  # for character model
          c_1 = id2char(np.random.randint(27, size=[1]))
          c_2 = id2char(np.random.randint(27, size=[1]))
          feed = np.array([dictionary[c_1+c_2]])  # for bigram model
          #sentence = id2char(feed)  # for character model
          sentence = c_1 + c_2
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.})
            pred_ohe = sample(prediction)  # get ohe of predicted proba
            pred_c = id2char(np.argmax(pred_ohe))  # convert id of prediction
            sentence += pred_c  # add predicted char
            feed = np.array([dictionary[c_2 + pred_c]])
            c_2 = pred_c
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        b_y = valid_batches_y.next()
        predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1.})
        valid_logprob = valid_logprob + logprob(predictions, b_y[2])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))
      print '-' * 30
# show how much time elapsed
print (time.time()-t0)/60., 'minutes elapsed'

Initialized
Average loss at step 0 = 3.30010414124 
learning rate: 10.0
Minibatch perplexity: 19.68
ilvkk bq lzicdl r snektotmryejbn ol  nndeyihp fctdeqpeijkc  hdeefrspywgl rq r uzm
xtltqprmq dz zedcernsx rnzmbvtyabhsweouxlyd rxdiwbdxdm  eoyicezoxxwauybqhda  pqtj
qv nthwqk ysv zkjgvooqskgseaj ij a astrxd sw lr am tj udeknn pfwpcpkitohvun tobeq
ibyvietdmhla fteetqgondnwnjsce cszpeerkvhe y y zaeowher u pndn cdehxpz reetbnbrhk
eqeirrp yij  otrvlefeysvccpyane a pspayei ot zurlemwoxpsyfssf kszapyyahlwgnrg cqr
Validation set perplexity: 19.57
------------------------------
Average loss at step 500 = 12.1783233047 
learning rate: 7.49894
Minibatch perplexity: 7.84
Validation set perplexity: 9.50
------------------------------
Average loss at step 1000 = 10.5631598008 
learning rate: 5.62341
Minibatch perplexity: 6.97
md thanksnugules cain fistions euling one sive nine eistions whoc he reprate six 
cglrive m fa nol and ected of eing as s sing a most of phony cumtimeled as jused 
ok condings sb

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [ ]:
%%bash
pwd
ls -l | wc -l

## Create mirrored text data

In [6]:
# look at text split into words
words = text.split()
words[:5]

['anarchism', 'originated', 'as', 'a', 'term']

In [68]:
# build a dictionary and replace rare words
import collections

vocabulary_size = 2**16  # orig 50000
words = text.split()

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()  # word: int
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))  # int: word
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print 'Most common words (+UNK) by count:'
print count[:15]
print 'Sample data (indexes of words):'
print data[:10]  # 
del words  # Hint to reduce memory.

Most common words (+UNK) by count:
[['UNK', 315138], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430), ('two', 192644), ('is', 183153), ('as', 131815), ('eight', 125285), ('for', 118445)]
Sample data (indexes of words):
[5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156]


In [11]:
# check if test sentence contains vocab words
test_text = 'the quick brown fox'
for i in test_text.split():
    print i, dictionary[i]

the 1
quick 3903
brown 1199
fox 2526


In [14]:
# build dictionary of reversed words
reversed_text = []
words = text.split()
for i in range(len(words)):
    reversed_text.append(words[i][::-1])
print 'Reversed text sample:', reversed_text[:5]

data_m, count_m, dictionary_m, reverse_dictionary_m = build_dataset(reversed_text)
print 'Most common words (+UNK) by count:'
print count_m[:15]
print 'Sample data (indexes of words):'
print data_m[:10]  # 
del words, reversed_text  # Hint to reduce memory.

Reversed text sample: ['msihcrana', 'detanigiro', 'sa', 'a', 'mret']
Most common words (+UNK) by count:
[['UNK', 315138], ('eht', 1061396), ('fo', 593677), ('dna', 416629), ('eno', 411764), ('ni', 372201), ('a', 325873), ('ot', 316376), ('orez', 264975), ('enin', 250430), ('owt', 192644), ('si', 183153), ('sa', 131815), ('thgie', 125285), ('rof', 118445)]
Sample data (indexes of words):
[5239, 3082, 12, 6, 195, 2, 3136, 46, 59, 156]


In [48]:
# check if test sentence contains vocab words
test_y = 'eht kciuq nworb xof'
for i in test_y.split():
    print i, dictionary_m[i]

eht 1
kciuq 3901
nworb 1199
xof 2527


### Seq2seq train and validation sets

In [61]:
valid_size_seq = 128
valid_data = data[:valid_size_seq]
train_data = data[valid_size_seq:]
valid_data_m = data_m[:valid_size_seq]
train_data_m = data_m[valid_size_seq:]
train_size_seq = len(train_data)
print train_size_seq, 'train size:\n', train_data[:64]
print valid_size_seq, 'valid size:\n', valid_data_m[:64]

17005079 train size:
[90, 27, 271, 38, 5954, 4870, 20522, 29, 57381, 42, 318, 6, 25772, 528, 7592, 372, 5, 259, 2, 154, 26, 1207, 12, 7592, 201, 1581, 3, 15341, 333, 1776, 7106, 4863, 345, 765, 161, 407, 5697, 756, 2, 4117, 1133, 4341, 1537, 3, 568, 8125, 99, 5239, 11, 52, 1409, 687, 19, 154, 27, 11, 156, 7106, 37, 2035, 1426, 8175, 2, 154]
128 valid size:
[5239, 3082, 12, 6, 195, 2, 3136, 46, 59, 156, 128, 742, 477, 10601, 134, 1, 27939, 2, 1, 103, 855, 3, 1, 15197, 61452, 2, 1, 151, 855, 3585, 1, 195, 11, 191, 59, 5, 6, 10755, 215, 7, 1326, 105, 455, 20, 59, 2734, 363, 7, 3675, 1, 709, 2, 372, 27, 41, 37, 54, 541, 98, 12, 6, 1426, 2761, 19]


## Batch generator for seq2seq

In [89]:
batch_size=64
num_unrollings=7

class BatchGeneratorSeq(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size / batch_size
    # list of offsets within batch
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size), dtype=np.int)  # 'id' of word to be embedded
    for b in xrange(self._batch_size):
      batch[b] = self._text[self._cursor[b]]  # add word 'id' to batch
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size  # move cursor
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array cooonsists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())  # add word 'id' for 1 to num_unrollings
    self._last_batch = batches[-1]
    return batches

def seqbatches2string(batches):
  """Convert a sequence of batches back into string
  representation.
  """
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [' '.join(x) for x in zip(s, [reverse_dictionary[c] for c in b])]  # use w/ character model
  return s

def seqbatches2string_m(batches):
  """Convert a sequence of batches back into string
  representation.
  """
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [' '.join(x) for x in zip(s, [reverse_dictionary_m[c] for c in b])]  # use w/ character model
  return s

In [90]:
# training and validation batches
train_batches = BatchGeneratorSeq(train_data, batch_size, num_unrollings)
valid_batches = BatchGeneratorSeq(valid_data, 1, 1) # returns batch size 1
train_batches_m = BatchGeneratorSeq(train_data_m, batch_size, num_unrollings)
valid_batches_m = BatchGeneratorSeq(valid_data_m, 1, 1) # returns batch size 1

# look at the text from various segments
segment_look = 0
show = segment_look * len(train_data)/batch_size
print "index {} to {}:\n{}".format(show, show+80, train_data[show:show+16])
print('-'*16)

train_X = train_batches.next()
print seqbatches2string(train_X)
print('-'*16)
val_X = valid_batches.next()
print val_X
print seqbatches2string(val_X)
print seqbatches2string(valid_batches.next())
print seqbatches2string_m(valid_batches_m.next())

index 0 to 80:
[90, 27, 271, 38, 5954, 4870, 20522, 29, 57381, 42, 318, 6, 25772, 528, 7592, 372]
----------------
[' use it does not imply chaos nihilism or', ' by assyrians around one two zero zero bc', ' sleeping murder having been resurrected from the fictional', ' five one nine zero seven located at four', ' the moors in the south afonso did not', ' he argued that there are certain rational laws', ' next season two zero zero five two zero', ' jena edited by m von rohr die UNK', ' both the general public and the legal profession', ' eight eight seven two nine zero three three', ' used to amplify the electric bass depending on', ' liberal he argues that sexual relationships outside of', ' zero th home run tying him with his', ' on october two five one nine nine nine', ' base crashed in montana all four members of', ' children in the early years young charles was', ' nitrogen sulfur and iron are burned they will', ' eight five two one nine zero two in', ' of going through this cycle 

## Build the seq2seq graph

In [31]:
# test sentence
test_batch_size = 1
test_unrollings = len(test_text)-1
test_batch = BatchGenerator(test_text, test_batch_size, test_unrollings)

a = test_batch.next()
print a[0].shape
print batches2string(a)

(1, 27)
['the quick brown fox']


In [43]:
# seq2seq word level lstm
num_nodes = 64
embedding_size = 128 # Dimension of the embedding vector.
vocabulary_size = len(count)

graph = tf.Graph()
with graph.as_default():
  
  ## Parameters:
  # combined i,f,c,o
  ifco_x = tf.Variable(tf.truncated_normal([4, embedding_size, num_nodes], -0.1, 0.1))
  print ifco_x.get_shape().as_list()
  ifco_m = tf.Variable(tf.truncated_normal([4, num_nodes, num_nodes], -0.1, 0.1))
  ifco_b = tf.Variable(tf.zeros([4, 1, num_nodes]))

  ifco_x2 = tf.Variable(tf.truncated_normal([4, embedding_size, num_nodes], -0.1, 0.1))
  ifco_m2 = tf.Variable(tf.truncated_normal([4, num_nodes, num_nodes], -0.1, 0.1))
  ifco_b2 = tf.Variable(tf.zeros([4, 1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_output2 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state2 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  test_saved_output = tf.Variable(tf.zeros([test_batch_size, num_nodes]), trainable=False)
  test_saved_state = tf.Variable(tf.zeros([test_batch_size, num_nodes]), trainable=False)
  test_saved_output2 = tf.Variable(tf.zeros([test_batch_size, num_nodes]), trainable=False)
  test_saved_state2 = tf.Variable(tf.zeros([test_batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, 1], -0.1, 0.1))
  b = tf.Variable(tf.zeros([1]))

  # Dropout
  #keep_prob = tf.placeholder(tf.float32)
    
  # Embedding Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), trainable=False)
  embeddings_m = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), trainable=False)
    
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """
    Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates.
    """                   
    i_list = tf.pack([i, i, i, i])
    o_list = tf.pack([o, o, o, o])
                          
    ins = tf.batch_matmul(i_list, ifco_x)
    outs = tf.batch_matmul(o_list, ifco_m)
    
    h_x = ins + outs + ifco_b
    
    forget_gate = tf.sigmoid(h_x[1,:,:])
    input_gate = tf.sigmoid(h_x[0,:,:])
    update = tf.tanh(h_x[2,:,:])
    
    state = forget_gate*state + input_gate*update
    output_gate = tf.sigmoid(h_x[3,:,:])
    
    h = output_gate * tf.tanh(state)
    return h, state

  def lstm_cell_2(i, o, state):
    """
    Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates.
    """                   
    i_list = tf.pack([i, i, i, i])
    o_list = tf.pack([o, o, o, o])
                          
    ins = tf.batch_matmul(i_list, ifco_x2)
    outs = tf.batch_matmul(o_list, ifco_m2)
    
    h_x = ins + outs + ifco_b
    
    forget_gate = tf.sigmoid(h_x[1,:,:])
    input_gate = tf.sigmoid(h_x[0,:,:])
    update = tf.tanh(h_x[2,:,:])
    
    state = forget_gate*state + input_gate*update
    output_gate = tf.sigmoid(h_x[3,:,:])
    
    h = output_gate * tf.tanh(state)
    return h, state

  # Input data: Encoder, Decoder
  train_data = list()  # Encoder input, len = num_unrollings+1
  train_data_dec = list()  # Decoder input, len = num_unrollings+2
  test_data = list()  # Decoder input, len = num_unrollings+2
    
  # append (or prepend) characters for each unrolling
  for _ in xrange(num_unrollings + 1):
    #train_data.append(
    #  tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
    train_data.insert(
      0, tf.placeholder(tf.float32, shape=[batch_size]))
    train_data_dec.append(
      tf.placeholder(tf.float32, shape=[batch_size]))

  for _ in xrange(test_unrollings+1):
    #test_data.append(
    #  tf.placeholder(tf.float32, shape=[test_batch_size, vocabulary_size]))
    test_data.insert(
      0, tf.placeholder(tf.float32, shape=[test_batch_size]))

  ###########################
  # TODO: Embedded input data
  emb_train_inputs = list()
  emb_train_dec_inputs = list()
  emb_test_inputs = list()
  for bigram in train_data:
    embed = tf.nn.embedding_lookup(embeddings, bigram)
    encoded_inputs.append(embed)
  train_inputs = encoded_inputs[:num_unrollings-1]

  train_inputs = train_data[:]
  #train_labels = train_data[1:]  # labels are inputs shifted by one time step.
  train_inputs_dec = train_data_dec[:num_unrollings]
  train_labels_dec = train_data_dec[:]  # labels are inputs shifted by one time step.
  test_inputs = test_data[:test_unrollings]

  ###########################

  # Unrolled LSTM loop, Encoder to Decoder
  outputs = list()  # keep Encoder outputs
  output = saved_output
  state = saved_state
  for i in train_inputs: # run the Encoder
    output, state = lstm_cell(i, output, state)
    outputs.append(output)
  outputs_dec = list()  # keep Decoder outputs
  output2 = saved_output2
  state2 = saved_state2
  go_input = []
  go_input.append(tf.Variable(tf.zeros([batch_size, vocabulary_size]), trainable=False))
  for i in go_input + train_inputs_dec:  # run the Decoder
    if i == go_input:  # use final Encoder state (and Encoder output?)
      output2, state2 = lstm_cell_2(i, output, state)
      #output2, state2 = lstm_cell_2(i, output2, state)
    else:
      output2, state2 = lstm_cell_2(i, output2, state2)
    outputs_dec.append(output2)
    
  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state),
                                saved_output2.assign(output2),
                                saved_state2.assign(state2)]):
    # Classifier.
    logits_en = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b) # outputs of Encoder
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs_dec), w, b)
    print 'logits', logits.get_shape().as_list()
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels_dec)))
    print 'labels', tf.concat(0, train_labels_dec).get_shape().as_list()
        
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=False)  ## orig 10.0, 5000, 0.1, True
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  print train_prediction.get_shape()

  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_output2 = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state2 = tf.Variable(tf.zeros([1, num_nodes]))

  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])),
    saved_sample_output2.assign(tf.zeros([1, num_nodes])),
    saved_sample_state2.assign(tf.zeros([1, num_nodes]))
    )
    
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)

  # go from Encoder to Decoder, pass Encoder state and output
  sample_output_go, sample_state_go = lstm_cell_2(
    tf.Variable(tf.zeros([1, vocabulary_size]), trainable=False), 
      saved_sample_output, saved_sample_state)
    
  sample_input_dec = tf.placeholder(tf.float32, shape=[1, vocabulary_size])  # Decoder input
  sample_output2, sample_state2 = lstm_cell_2(
    sample_input_dec, saved_sample_output2, saved_sample_state2)
    
  #################################  
  # TODO: translate a test sentence
  test_outputs_en = list()  # keep Encoder outputs
  test_outputs = list()  # keep Decoder outputs
  test_output = test_saved_output
  test_state = test_saved_state
  test_output2 = test_saved_output2
  test_state2 = test_saved_state2

  for i in test_inputs: # run the Encoder
    test_output, test_state = lstm_cell(i, test_output, test_state)
    test_outputs_en.append(test_output)
  go_input = []
  go_input.append(tf.Variable(tf.zeros([test_batch_size, vocabulary_size]), trainable=False))
  for i in go_input + test_inputs:  # run the Decoder
    logits = tf.nn.xw_plus_b(test_output2, w, b)
    test_input2 = tf.nn.softmax(logits)
    if i == go_input:  # use final Encoder state (and output?)
      test_output2, test_state2 = lstm_cell_2(i, test_output, test_state)
      #test_output2, test_state2 = lstm_cell_2(i, test_output2, test_state)
    else:
      test_output2, test_state2 = lstm_cell_2(test_input2, test_output2, test_state2)
    test_outputs.append(test_output2)
  
  with tf.control_dependencies([test_saved_output.assign(test_output),
                                test_saved_state.assign(test_state),
                                test_saved_output2.assign(test_output2),
                                test_saved_state2.assign(test_state2)]):
    test_logits_en = tf.nn.xw_plus_b(tf.concat(0, test_outputs_en), w, b) # outputs of Encoder
    test_prediction = tf.nn.softmax(tf.nn.xw_plus_b(tf.concat(0, test_outputs), w, b))
    print test_prediction.get_shape().as_list()
  ##################################

  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state),
                                saved_sample_output2.assign(sample_output2),
                                saved_sample_state2.assign(sample_state2)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output2, w, b))

  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state),
                                saved_sample_output2.assign(sample_output2),
                                saved_sample_state2.assign(sample_state2)]):
    sample_prediction_go = tf.nn.softmax(tf.nn.xw_plus_b(sample_output_go, w, b))
    sample_output2, sample_state2 = sample_output_go, sample_state_go # pass state, output
    


[4, 27, 64]


TypeError: DataType float32 for attr 'Tindices' not in list of allowed values: int32, int64

## Run the seq2seq

In [424]:
# seq2seq training and validation batches
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1) # returns batch size 1, +1 unrolling
train_batches_m = BatchGenerator(train_mtext, batch_size, num_unrollings)
valid_batches_m = BatchGenerator(valid_mtext, 1, 1) # returns batch size 1, +2 unrolling

# character LSTM
num_steps = 7001  ## orig 7001
summary_frequency = 100

t0 = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized\n=========='
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    batches_m = train_batches_m.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
      feed_dict[train_data_dec[i]] = batches_m[i]

    _, _, l, predictions, lr = session.run(
      [optimizer, logits_en, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % (5.*summary_frequency) == 0:  ## orig 2.5*summary_frequency
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step ', step, '=', mean_loss, '\nlearning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches_m)[:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          print 'Start:', characters(feed)[0]
          sentence = ''
          reset_sample_state.run()
          prediction = sample_prediction_go.eval(
            {sample_input: feed, sample_input_dec: feed})
          feed = sample(prediction)
          sentence += characters(feed)[0]
          for _ in xrange(63):
            prediction = sample_prediction.eval(
              {sample_input: feed, sample_input_dec: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        b_m = valid_batches_m.next()
        predictions = sample_prediction_go.eval(
          {sample_input: b[0], sample_input_dec: np.zeros([1,vocabulary_size])})
        valid_logprob = valid_logprob + logprob(predictions, b_m[0])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))
      
      #####################################
      # TODO: predict 'the quick brown fox'
      batches_t = test_batch.next()
      feed_dict = dict()
      for i in xrange(test_unrollings+1):
        feed_dict[test_data[i]] = batches_t[i]
      _, prediction = session.run([test_logits_en, test_prediction], feed_dict=feed_dict)
      sentence = ''
      for i in characters(prediction):
        sentence += i
      print 'test:', sentence, '(end)'
      #####################################
    
      print '-' * 30
# show how much time elapsed
print (time.time()-t0)/60., 'minutes elapsed'

Initialized
Average loss at step  0 = 3.29499197006 
learning rate: 10.0
Minibatch perplexity: 26.98
Start: k
df jpdkqcte aouk lrqg rtsnk peseeoe f qrtxctc  aeww prkehmtftw t
Start: v
ayxc loaopjaliehfta  pf aafbkenxwlg g axznxletfpmb t eisteklponr
Start: x
wnraa syelocsitsrr pcsp  e   tswhm fyoypccsejfjrboaarfemttd  sny
Start: l
sbmse ikhti  ooivnt i zvhlwnyaexptuttssoqpueya aixoza x ki ee mo
Start: z
hti zljtp fxyirsieqs gvzm emgdimenusryahb veaxtbx surrjtducdtnai
Validation set perplexity: 20.26
test:                     (end)
------------------------------
Average loss at step  500 = 11.9628528702 
learning rate: 7.94328
Minibatch perplexity: 8.71
Validation set perplexity: 20.62
test: ee                  (end)
------------------------------
Average loss at step  1000 = 10.0397679472 
learning rate: 6.30957
Minibatch perplexity: 6.91
Start: x
lif semil sa lairetu fo rehto desah senitrevod eht yroj deccepem
Start: s
retom eht tnemra etan fo nac yeht erevos rewohcar s niaetimom eh
St

---

## Scratchpad

In [19]:
# build text with mirrored words
reversed_text = []
words = text.split()
for i in range(len(words)):
    reversed_text.append(words[i][::-1])
print 'Reversed text sample:', reversed_text[:5]

mirrored_text = ' ' + ' '.join(reversed_text)
print mirrored_text[:200]
del words, reversed_text

Reversed text sample: ['msihcrana', 'detanigiro', 'sa', 'a', 'mret']


In [21]:
valid_mtext = mirrored_text[:valid_size]
train_mtext = mirrored_text[valid_size:]
train_size = len(train_mtext)
print train_size, train_mtext[:64]
print valid_size, valid_mtext[:64]

99999000 sni stsihcrana etacovda laicos snoitaler desab nopu yratnulov no
1000  msihcrana detanigiro sa a mret fo esuba tsrif desu tsniaga ylra


In [39]:
# Use character model batch generators with mirrored text
# training and validation batches
num_unrollings = 15
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1) # returns batch size 1
train_batches_m = BatchGenerator(train_mtext, batch_size, num_unrollings)
valid_batches_m = BatchGenerator(valid_mtext, 1, 1) # returns batch size 1

# look at the text from various segments
segment_look = 0
show = segment_look * len(train_text)/batch_size
print "index {} to {}:\n{}".format(show, show+80, train_text[show:show+64])
print('-'*16)

train_X = train_batches.next()
print batches2string(train_X)
print batches2string(train_batches_m.next())
print('-'*16)
val_X = valid_batches.next()
print val_X
print batches2string(val_X)
#print valid_batches_y.next()
print batches2string(valid_batches.next())
print batches2string(valid_batches_m.next())
print batches2string(valid_batches_m.next())

index 0 to 80:
ons anarchists advocate social relations based upon voluntary as
----------------
['ons anarchists a', 'lleria arches na', 'married urraca p', 'y and liturgical', 'tion from the na', 'new york other w', 'e listed with a ', 'o be made to rec', 'ore significant ', ' two six eight i', 'ity can be lost ', 'tion of the size', 'f certain drugs ', 'e convince the p', 'ampaign and barr', 'ious texts such ', 'a duplicate of t', 'ine january eigh', 'cal theories cla', ' dimensional ana', 't s support or a', 'e oscillating sy', 'of italy languag', 'klahoma press on', 'ws becomes the f', 'the fabian socie', ' sharman network', 'ting in politica', 'th risky riskerd', 'fense the air co', 'treet grid cente', 'appeal of devoti']
['sni stsihcrana e', 'rellag sehcra la', 'deirram acarru s', 'l dna lacigrutil', 'etta morf eht la', 'wen kroy rehto l', 'a detsil htiw a ', 'a eb edam ot ezi', 'rom tnacifingis ', ' owt xis thgie n', 'tan nac eb tsol ', 'cnuf fo eht ezis', 'o niatrec sgurd ', '

In [46]:
# Build the graph
# seq2seq character lstm
num_nodes = 128
batch_size=32
num_unrollings=15

graph = tf.Graph()
with graph.as_default():
  
  ## Parameters:
  # combined i,f,c,o
  ifco_x = tf.Variable(tf.truncated_normal([4, vocabulary_size, num_nodes], -0.1, 0.1))
  print ifco_x.get_shape().as_list()
  ifco_m = tf.Variable(tf.truncated_normal([4, num_nodes, num_nodes], -0.1, 0.1))
  ifco_b = tf.Variable(tf.zeros([4, 1, num_nodes]))

  ifco_x2 = tf.Variable(tf.truncated_normal([4, vocabulary_size, num_nodes], -0.1, 0.1))
  ifco_m2 = tf.Variable(tf.truncated_normal([4, num_nodes, num_nodes], -0.1, 0.1))
  ifco_b2 = tf.Variable(tf.zeros([4, 1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_output2 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state2 = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  test_saved_output = tf.Variable(tf.zeros([test_batch_size, num_nodes]), trainable=False)
  test_saved_state = tf.Variable(tf.zeros([test_batch_size, num_nodes]), trainable=False)
  test_saved_output2 = tf.Variable(tf.zeros([test_batch_size, num_nodes]), trainable=False)
  test_saved_state2 = tf.Variable(tf.zeros([test_batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Dropout
  #keep_prob = tf.placeholder(tf.float32)
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """
    Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates.
    """                   
    i_list = tf.pack([i, i, i, i])
    o_list = tf.pack([o, o, o, o])
                          
    ins = tf.batch_matmul(i_list, ifco_x)
    outs = tf.batch_matmul(o_list, ifco_m)
    
    h_x = ins + outs + ifco_b
    
    forget_gate = tf.sigmoid(h_x[1,:,:])
    input_gate = tf.sigmoid(h_x[0,:,:])
    update = tf.tanh(h_x[2,:,:])
    
    state = forget_gate*state + input_gate*update
    output_gate = tf.sigmoid(h_x[3,:,:])
    
    h = output_gate * tf.tanh(state)
    return h, state

  def lstm_cell_2(i, o, state):
    """
    Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates.
    """                   
    i_list = tf.pack([i, i, i, i])
    o_list = tf.pack([o, o, o, o])
                          
    ins = tf.batch_matmul(i_list, ifco_x2)
    outs = tf.batch_matmul(o_list, ifco_m2)
    
    h_x = ins + outs + ifco_b
    
    forget_gate = tf.sigmoid(h_x[1,:,:])
    input_gate = tf.sigmoid(h_x[0,:,:])
    update = tf.tanh(h_x[2,:,:])
    
    state = forget_gate*state + input_gate*update
    output_gate = tf.sigmoid(h_x[3,:,:])
    
    h = output_gate * tf.tanh(state)
    return h, state

  # Input data: Encoder, Decoder
  train_data = list()  # Encoder input, len = num_unrollings+1
  train_data_dec = list()  # Decoder input, len = num_unrollings+2
  test_data = list()  # Decoder input, len = num_unrollings+2

  # append (or prepend) characters for each unrolling
  for _ in xrange(num_unrollings + 1):
    #train_data.append(
    #  tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
    train_data.insert(
      0, tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
    train_data_dec.append(
      tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))

  for _ in xrange(test_unrollings+1):
    #test_data.append(
    #  tf.placeholder(tf.float32, shape=[test_batch_size, vocabulary_size]))
    test_data.insert(
      0, tf.placeholder(tf.float32, shape=[test_batch_size, vocabulary_size]))
    
  train_inputs = train_data[:]
  #train_labels = train_data[1:]  # labels are inputs shifted by one time step.
  train_inputs_dec = train_data_dec[:num_unrollings]
  train_labels_dec = train_data_dec[:]  # labels are inputs shifted by one time step.
  test_inputs = test_data[:test_unrollings]

  # Unrolled LSTM loop, Encoder to Decoder
  outputs = list()  # keep Encoder outputs
  output = saved_output
  state = saved_state
  for i in train_inputs: # run the Encoder
    output, state = lstm_cell(i, output, state)
    outputs.append(output)
  outputs_dec = list()  # keep Decoder outputs
  output2 = saved_output2
  state2 = saved_state2
  go_input = []
  go_input.append(tf.Variable(tf.zeros([batch_size, vocabulary_size]), trainable=False))
  for i in go_input + train_inputs_dec:  # run the Decoder
    if i == go_input:  # use final Encoder state (and Encoder output?)
      output2, state2 = lstm_cell_2(i, output, state)
      #output2, state2 = lstm_cell_2(i, output2, state)
    else:
      output2, state2 = lstm_cell_2(i, output2, state2)
    outputs_dec.append(output2)
    
  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state),
                                saved_output2.assign(output2),
                                saved_state2.assign(state2)]):
    # Classifier.
    logits_en = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b) # outputs of Encoder
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs_dec), w, b)
    print 'logits', logits.get_shape().as_list()
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels_dec)))
    print 'labels', tf.concat(0, train_labels_dec).get_shape().as_list()
        
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=False)  ## orig 10.0, 5000, 0.1, True
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  print train_prediction.get_shape()

  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_output2 = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state2 = tf.Variable(tf.zeros([1, num_nodes]))

  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])),
    saved_sample_output2.assign(tf.zeros([1, num_nodes])),
    saved_sample_state2.assign(tf.zeros([1, num_nodes]))
    )
    
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)

  # go from Encoder to Decoder, pass Encoder state and output
  sample_output_go, sample_state_go = lstm_cell_2(
    tf.Variable(tf.zeros([1, vocabulary_size]), trainable=False), 
      saved_sample_output, saved_sample_state)
    
  sample_input_dec = tf.placeholder(tf.float32, shape=[1, vocabulary_size])  # Decoder input
  sample_output2, sample_state2 = lstm_cell_2(
    sample_input_dec, saved_sample_output2, saved_sample_state2)
    
  #################################  
  # TODO: translate a test sentence
  test_outputs_en = list()  # keep Encoder outputs
  test_outputs = list()  # keep Decoder outputs
  test_output = test_saved_output
  test_state = test_saved_state
  test_output2 = test_saved_output2
  test_state2 = test_saved_state2

  for i in test_inputs: # run the Encoder
    test_output, test_state = lstm_cell(i, test_output, test_state)
    test_outputs_en.append(test_output)
  go_input = []
  go_input.append(tf.Variable(tf.zeros([test_batch_size, vocabulary_size]), trainable=False))
  for i in go_input + test_inputs:  # run the Decoder
    logits = tf.nn.xw_plus_b(test_output2, w, b)
    test_input2 = tf.nn.softmax(logits)
    if i == go_input:  # use final Encoder state (and output?)
      test_output2, test_state2 = lstm_cell_2(i, test_output, test_state)
      #test_output2, test_state2 = lstm_cell_2(i, test_output2, test_state)
    else:
      test_output2, test_state2 = lstm_cell_2(test_input2, test_output2, test_state2)
    test_outputs.append(test_output2)
  
  with tf.control_dependencies([test_saved_output.assign(test_output),
                                test_saved_state.assign(test_state),
                                test_saved_output2.assign(test_output2),
                                test_saved_state2.assign(test_state2)]):
    test_logits_en = tf.nn.xw_plus_b(tf.concat(0, test_outputs_en), w, b) # outputs of Encoder
    test_prediction = tf.nn.softmax(tf.nn.xw_plus_b(tf.concat(0, test_outputs), w, b))
    print test_prediction.get_shape().as_list()
  ##################################

  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state),
                                saved_sample_output2.assign(sample_output2),
                                saved_sample_state2.assign(sample_state2)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output2, w, b))

  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state),
                                saved_sample_output2.assign(sample_output2),
                                saved_sample_state2.assign(sample_state2)]):
    sample_prediction_go = tf.nn.softmax(tf.nn.xw_plus_b(sample_output_go, w, b))
    sample_output2, sample_state2 = sample_output_go, sample_state_go # pass state, output
    


[4, 27, 128]
logits [512, 27]
labels [512, 27]
(512, 27)
[19, 27]


### Run character LSTM

In [47]:
# Run the seq2seq character lstm
# training and validation batches
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1) # returns batch size 1, +1 unrolling
train_batches_m = BatchGenerator(train_mtext, batch_size, num_unrollings)
valid_batches_m = BatchGenerator(valid_mtext, 1, 1) # returns batch size 1, +2 unrolling

# character LSTM
num_steps = 7001  ## orig 7001
summary_frequency = 100

t0 = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print 'Initialized\n=========='
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    batches_m = train_batches_m.next()
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
      feed_dict[train_data_dec[i]] = batches_m[i]

    _, _, l, predictions, lr = session.run(
      [optimizer, logits_en, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % (5.*summary_frequency) == 0:  ## orig 2.5*summary_frequency
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print 'Average loss at step ', step, '=', mean_loss, '\nlearning rate:', lr
      mean_loss = 0
      labels = np.concatenate(list(batches_m)[:])
      print 'Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels)))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print '=' * 80
        for _ in xrange(5):
          feed = sample(random_distribution())
          print 'Start:', characters(feed)[0]
          sentence = ''
          reset_sample_state.run()
          prediction = sample_prediction_go.eval(
            {sample_input: feed, sample_input_dec: feed})
          feed = sample(prediction)
          sentence += characters(feed)[0]
          for _ in xrange(63):
            prediction = sample_prediction.eval(
              {sample_input: feed, sample_input_dec: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print sentence
        print '=' * 80
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        b_m = valid_batches_m.next()
        predictions = sample_prediction_go.eval(
          {sample_input: b[0], sample_input_dec: np.zeros([1,vocabulary_size])})
        valid_logprob = valid_logprob + logprob(predictions, b_m[0])
      print 'Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size))
      
      #####################################
      # TODO: predict 'the quick brown fox'
      batches_t = test_batch.next()
      feed_dict = dict()
      for i in xrange(test_unrollings+1):
        feed_dict[test_data[i]] = batches_t[i]
      _, prediction = session.run([test_logits_en, test_prediction], feed_dict=feed_dict)
      sentence = ''
      for i in characters(prediction):
        sentence += i
      print 'test:', sentence, '(end)'
      #####################################
    
      print '-' * 30
# show how much time elapsed
print (time.time()-t0)/60., 'minutes elapsed'

Initialized
Average loss at step  0 = 3.30632829666 
learning rate: 10.0
Minibatch perplexity: 27.28
Start: f
wallsbqfpua xshwkv nimiucefyxezjr gadrzova oa  iuco byic qyoig a
Start: e
ssif thwn bmjktr  pdpwbwn o mruqlanb dinfnrtxurnecgu hllubdtgl  
Start: g
rz xoln  dmtqleyd atsetmrnll e lexthftftrtni alismcweza texopozc
Start: c
cm xsrplfrhl zkbim j   g r fifyepyuaatmrmitbdfbadifczpdbajsraeeo
Start: f
ktepqmderuwz  wllf i  nstukesfegrdhflp ijaviclerntg zcswvaufxip 
Validation set perplexity: 20.00
test:                     (end)
------------------------------
Average loss at step  500 = 11.6222466779 
learning rate: 7.94328
Minibatch perplexity: 8.51
Validation set perplexity: 21.60
test: eeeee e eee eee e e (end)
------------------------------
Average loss at step  1000 = 9.79467670798 
learning rate: 6.30957
Minibatch perplexity: 6.47
Start: r
noommu dorreta rof srevalgnuk gnol egaz ecif a ralupma eht eno e
Start: g
 telbhotsap sevallak reva evistniw seht esihc hsiwelbap deralle 
St